In [112]:
!pip install litellm

In [ ]:
import os
# from google.colab import userdata
# api_key = userdata.get('OPENAI_API_KEY')

api_key = os.getenv("OPENAI_API_KEY")
# print(f"API KEY: {api_key}")

In [10]:
from litellm import completion
from typing import List, Dict
import json

In [11]:
def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content

In [12]:
def extract_code_block(response: str) -> str:
    """Extract code block from response"""
    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()
    if code_block.startswith("python"):
        code_block = code_block[6:]

    return code_block

In [62]:
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:
- list_files() -> List[str]: List all files in the current directory.
- read_file(file_name: str) -> str: Read the content of a file.
- terminate(message: str): End the agent loop and print a summary to the user.

If a user asks about files, list them before reading.

Every returned response MUST have an action.
Every response MUST be in this format:
```
{
    "action": {
        "tool_name": "insert tool_name",
        "args": {
            ...fill in any required arguments here...
        }
    }
}
```
"""
}]

In [46]:
memory = [
    {"role": "user", "content": "What files are in this directory?"},
    {"role": "assistant", "content": "```\"action\":\n{\"tool_name\":\"list_files\",\"args\":{}}\n```"},
    {"role": "user", "content": "[\"test_files/file1.txt\", \"test_files/file2.txt\"]"}
]

In [ ]:
def extract_markdown_block(response: str):
    """Parse the action from the response."""
    # Assuming the action is the first word in the response
    """Extract code block from response"""
    if not '```json' in response:
        return response

    code_block = response.split('```json')[1].strip()

    # strip the last 3 characters
    parse_block = json.loads(code_block[:-3])
    return parse_block["action"]


def parse_action(response: str) -> Dict:
    """Parse the LLM response into a structured action dictionary."""
    try:
        response_json = extract_markdown_block(response)
        # print(f"response : {response_json["tool_name"]}")
        if response_json["tool_name"] and response_json["args"]:
            print(f"json response : {print(response_json)}")
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        return {"tool_name": "Unable to Parse", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}

In [108]:
def list_files():
    return os.listdir()

def read_file(action):
    return "Files read"

In [ ]:
# The Agent Loop
iterations = 0
max_iterations = 3
while iterations < max_iterations:

    # 1. Construct prompt: Combine agent rules with memory
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")

    # 3. Parse response to determine action
    action = parse_action(response)
    print(f"action: {action}")
    result = "Action executed"
    # print(f"Action Tool Name: {action["tool_name"]}")
    print(f"Action: {action["tool_name"]}")
    if action["tool_name"] == "list_files":
        result = {"result":list_files()}
    elif action["tool_name"] == "read_file":
        result = {"result":read_file(action["args"]["file_name"])}
    elif action["tool_name"] == "error":
        result = {"error":action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break
    else:
        result = {"error":"Unknown action: "+action["tool_name"]}

    print(f"Action result: {result}")

    # 5. Update memory with response and results
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])

    # 6. Check termination condition
    if action["tool_name"] == "terminate":
        print(f"tool_name : {action['tool_name']}")
        break

    iterations += 1

Agent thinking...
Agent response: ```json
{
    "action": {
        "tool_name": "terminate",
        "args": {
            "message": "I have listed and attempted to read the files: 'test_files/file1.txt' and 'test_files/file2.txt'."
        }
    }
}
```
{'tool_name': 'terminate', 'args': {'message': "I have listed and attempted to read the files: 'test_files/file1.txt' and 'test_files/file2.txt'."}}
json response : None
action: {'tool_name': 'terminate', 'args': {'message': "I have listed and attempted to read the files: 'test_files/file1.txt' and 'test_files/file2.txt'."}}
Action: terminate
I have listed and attempted to read the files: 'test_files/file1.txt' and 'test_files/file2.txt'.
